## Merge Primary and Secondary Dataset

In [21]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import re

In [33]:
# turn on debug mode when running by itself
if not 'debug' in locals():
    debug = True
#debug=False

In [34]:
# load primary dataset
df_all=pd.read_csv('20210412020005_covid-19.tsv',sep='\t')

# filter clinical trial dataset to industry company 
# since industry company are the only possible public traded sponsors, rest are institutions
df=df_all[df_all["funded_bys"].str.find('Industry')>=0]


In [35]:
# load secondary dataset
# download from https://www.nasdaq.com/market-activity/stocks/screener
#ticker_all=pd.read_csv('nasdaq_screener_healthcare.csv') # healthcare sector only
ticker_all=pd.read_csv('nasdaq_stocks.csv')
ticker_all=ticker_all[['Symbol','Name','Sector']][ticker_all.Sector != 'Technology']
ticker_all['Name']=ticker_all['Name'].str.lower()

In [36]:
if (debug):
    print("origin clinical data shape: ", df_all.shape)
    print("after filter to industry company data shape: ", df.shape)
    print("ticker data shape: ", ticker_all.shape)

In [37]:
# get industry participant and clean company name to keep key words in the name
def getCompanyName(sponsor_collaborators, lead, collaborators):
    
    sponsors = sponsor_collaborators.split('|')
    company = ""
    for sponsor in sponsors: # can be more than 1 industry companies
        if (lead.find(sponsor+"[Industry]")>=0):
            company = company + "|" + sponsor
        elif (collaborators != None) & (collaborators.find(sponsor+"[Industry]")>=0): #collaborators can be NaN
            company = company + "|" + sponsor
    
    company = company.replace('Sanofi Pasteur, a Sanofi Company', 'Sanofi').replace('Aevi Genomic Medicine a Cerecor', 'Cerecor') \
                    .replace('Janssen Vaccines','Johnson & Johnson').replace('Janssen Research','Johnson & Johnson') \
                    .replace('Janssen Pharmaceutica','Johnson & Johnson').replace('Janssen Pharmaceutical','Johnson & Johnson') \
                    .replace('Amarin Pharma', 'Amarin').replace('Aytu BioScience','aytu biopharma').replace('Technologies B.V.','') \
                    .replace('Rostock','').replace('Bellerophon Pulse Technologies','bellerophon therapeutics') \
                    .replace(', Inc.','').replace(', Inc','').replace('. Inc.','').replace('Inc.,','').replace('Inc.','') \
                    .replace('Incorporated','').replace('Corporation','').replace('Corp','').replace('Company','').replace('company','')    \
                    .replace("Co., Ltd.","").replace("Co.,", "").replace("Co.", "").replace(", S.A.", "").replace('Partners','') \
                    .replace('Pty Ltd','').replace(", Ltd.","").replace("Ltd.", "").replace(', LLC','').replace('LLC','') \
                    .replace('Austria GmbH','').replace('RNA Pharmaceuticals GmbH','').replace('GmbH','').replace(" AG", "") \
                    .replace('North America','').replace('Americas','').replace('International','').replace('International Sàrl','') \
                    .replace('Biopharmaceuticals','').replace('Biopharma','').replace('BioScience','').replace('biotech','').replace('bio','') \
                    .replace('Therapeutics','').replace('healthcare','').replace('HealthCare','').replace(' Health Inc','').replace(' Health','') \
                    .replace('scientific','').replace('Clinical Research','').replace('Nanomedicine','').replace('Oncology','').replace('Pharma  ','') \
                    .replace(",", "")
        
    return company[1:]

In [38]:
# getCompanyName testing
if (debug):
    s1='Han Xu, M.D., Ph.D., Sponsor-Investigator, IRB Chair|PPD'
    s2='Han Xu, M.D., Ph.D., Sponsor-Investigator, IRB Chair[Industry]'
    s3='PPD[Industry]'
    # s1='Biohaven Pharmaceuticals, Inc.'
    # s2='Biohaven Pharmaceuticals, Inc.[Industry]'
    # s3=''
    print(getCompanyName(s1, s2, s3))

In [39]:
# find stock symbols
# input: multiple company names, nasdaq all company symbol and names
# output: matched symbol list
def findSymbol(companies, tickers):
    result=[]
    for company in companies.split('|'): #can be more than 1
        company = company.lower()
        tmp = tickers[['Symbol']][tickers.Name.str.find(company)==0]
        if len(tmp.Symbol) >= 1: # find the whole name      
            result.append(tmp.Symbol.values[0])
        else:  # didn't find with the whole name
            words = company.split(' ')
            if len(words) > 2: # if company name have more than 2 words, use 2 words
                name = words[0] + ' ' + words[1]
                tmp = tickers[['Symbol']][tickers.Name.str.find(name)==0]
                if len(tmp.Symbol) >= 1:
                    result.append(tmp.Symbol.values[0])
            elif (len(words) == 2):
                if (words[1]=='pharmaceuticals'): # if company name has 2 words like [**] pharmaceutical
                    name = words[0] + ' pharmaceutical' # remove s from Pharmaceuticals for BHVN
                    tmp = tickers[['Symbol']][tickers.Name.str.find(name)==0]
                    if len(tmp.Symbol) >= 1:
                        result.append(tmp.Symbol.values[0])
                    else:  #if 2 words name not found, try to drop pharmaceutical
                        name = words[0]  # use just the first word
                        tmp = tickers[['Symbol']][tickers.Name.str.find(name)==0]
                        if len(tmp.Symbol) >= 1:
                            result.append(tmp.Symbol.values[0])
    
    return result

In [40]:
# findSymbol testing
if (debug):
    s='IRB Chair|Adamis Pharmaceutical|CureVac'
    print(findSymbol(s, ticker_all))    

In [41]:
#
# merge 2 dataset based on company name
#
# industry company name can be either in lead_sponsor or collaborators
df['company']=df.apply(lambda x: getCompanyName(x.sponsor_collaborators, x.lead_sponsor, x.collaborators), axis=1)

# find stock symbol based on company name
df['symbol']=df.company.apply(lambda x: findSymbol(x, ticker_all))

# since trials can involve multiple public companies
df=df.explode('symbol')


<ipython-input-41-11ba52ca5917>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['company']=df.apply(lambda x: getCompanyName(x.sponsor_collaborators, x.lead_sponsor, x.collaborators), axis=1)
<ipython-input-41-11ba52ca5917>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['symbol']=df.company.apply(lambda x: findSymbol(x, ticker_all))


In [42]:
# initial merge result check
if (debug):
    print('data shape after merge: ', df.shape)
    # check empty company names
    print('no company name data shape:',df[df['company']==''].shape)
    print(df[['nct_id', 'lead_sponsor','collaborators', 'company']][df['company']==''].shape)
    
    # check the list of companies have no symbol
    print('symbol not found data shape:',df[pd.isna(df.symbol)].shape)
    print(df[['nct_id', 'lead_sponsor','company', 'symbol']][pd.isna(df.symbol)])


In [43]:
# spot check companies have no symbol
if (debug):
    #print(df[['nct_id','sponsor_collaborators', 'lead_sponsor','collaborators','company', 'symbol']][df.company.str.find('Sanofi')==0])    
    print(df[['nct_id', 'funded_bys', 'sponsor_collaborators', 'lead_sponsor','collaborators', 'company','symbol']][df.company.str.find('Amarin')>=0])
    print(df[['nct_id', 'funded_bys', 'sponsor_collaborators', 'lead_sponsor','collaborators', 'company','symbol']][df.nct_id=='NCT04346615'])


In [44]:
# double check: for company didn't find symbol, try again if any part of the name matched a public company name
if (debug):
    cnlist = df['company'][pd.isna(df.symbol)].unique().tolist()
    exceptwordlist = ['a','ai','ab','d/b/a','co','de','et','e','s','sl','inc','sa', 'sas','ve','bio','la','lp','and','the','first','of', 'group', \
                  'm.d.','md','dr.','research','medical','life','world', 'human','tree','plant','nature','stock','therapeutic',\
                  'health','science','scientific','partner','center','stem','cell','biotech','biotec', \
                  'global','national','europe','british','canada','us','usa','india','pacific','federal','east','middle']
    for c in cnlist:    
        words = c.split(' ')
        for aword in words:
            aword = aword.lower()
            if (len(aword)>0) & (aword not in exceptwordlist):
                tmp = ticker_all[['Symbol', 'Name']][ticker_all.Name.str.find(aword)==0]
                if len(tmp.Symbol) >= 1:
                    print('COMPANY(', c, ') WITH NAME PART(', aword, ') MATCH TICKER(', tmp.Symbol.values[0] + ') NAMED-', tmp.Name.values[0])


In [45]:
# compare our symbol lists
if (debug):
    print('total unique symbol:', df['symbol'].nunique(), df['symbol'].unique())
    xlist=df['symbol'].unique()
    #wlist=['PPD', 'SNY', 'CDNA', 'BLPH', 'PFE', 'GSK', 'BGNE', 'CMRX', 'FGEN', 'TAK', 'BMRA', 'AZN', 'VXRT', 'HRC', 'NVAX', 'AMGN', 'CFRX', 'EQ', 'ABBV', 'PRTC', 'HSTO', 'GILD', 'REGN', 'VNDA', 'LLY', 'BHVN', 'KPTI', 'TBPH', 'HGEN', 'ALXN', 'INCY', 'BHC', 'MRKR', 'LBPS', 'ATHX', 'BNTX', 'IBRX', 'VERU', 'WINT', 'EXAS', 'AVIR', 'XAIR', 'EDSA', 'APLS', 'VTGN', 'CTIC', 'LIFE', 'KLDO', 'SNDX', 'PHAS', 'PTCT', 'SRNE', 'ALT', 'KNSA', 'AMPE', 'CRVS', 'HRTX', 'BMY', 'ARCT', 'EVLO', 'ADPT', 'FULC', 'CLBS', 'VRNA', 'SAGE', 'VIR', 'KMDA', 'OPK', 'GRTX', 'PLRX', 'DFFN', 'NGM', 'AEMD', 'NRBO', 'HUGE', 'RIGL', 'INO', 'NKTR', 'ICLR', 'ENLV', 'TLC', 'LHDX', 'ADMP', 'VBLT', 'VBIV', 'BDX', 'RDY', 'JNJ']
    #wlist=['PPD', 'SNY', 'CDNA', 'BLPH', 'PFE', 'GSK', 'BGNE', 'CMRX', 'MNOV', 'FGEN', 'DRRX', 'TAK', 'BMRA', 'AZN', 'VXRT', 'HRC', 'NVAX', 'AMGN', 'CFRX', 'EQ', 'ABBV', 'PRTC', 'HSTO', 'GILD', 'REGN', 'VNDA', 'LLY', 'BHVN', 'KPTI', 'TBPH', 'HGEN', 'ALXN', 'INCY', 'BHC', 'MRKR', 'LBPS', 'ATHX', 'BNTX', 'INMB', 'IBRX', 'VERU', 'WINT', 'EXAS', 'AVIR', 'XAIR', 'EDSA', 'APLS', 'VTGN', 'CTIC', 'LIFE', 'KLDO', 'RDHL', 'SNDX', 'PHAS', 'PTCT', 'SRNE', 'ALT', 'KNSA', 'AMPE', 'CRVS', 'HRTX', 'BMY', 'ARCT', 'EVLO', 'ADPT', 'FULC', 'CLBS', 'VRNA', 'SAGE', 'VIR', 'KMDA', 'OPK', 'GRTX', 'PLRX', 'DFFN', 'NGM', 'AEMD', 'NRBO', 'HUGE', 'RIGL', 'INO', 'NKTR', 'ICLR', 'ENLV', 'TLC', 'LHDX', 'ADMP', 'VBLT', 'VBIV', 'BDX', 'RDY', 'JNJ']
    wlist = ['PPD', 'NVS', 'BCRX', 'GILD', 'REGN', 'SNY', 'DGX', 'VNDA', 'ABBV', 'PFE', 'CANF', 'INO', 'TRHC', 'CDNA', 'LLY', 'EIGR', 'AZN', 'BHVN', 'BMY', 'TBPH', 'HGEN', 'UNH', 'INCY', 'ALXN', 'MRKR', 'BLPH', 'LBPS', 'MEDP', 'ATHX', 'BNTX', 'NVAX', 'IPHA', 'INMB', 'MESO', 'VSTM', 'GSK', 'IMUX', 'AIM', 'ECOR', 'BGNE', 'IBRX', 'VERU', 'PSTI', 'WINT', 'CMRX', 'CTSO', 'EXAS', 'KNSA', 'ALVR', 'APLS', 'VTGN', 'CTIC', 'LIFE', 'KLDO', 'RDHL', 'SNDX', 'TAK', 'MNOV', 'FGEN', 'PHAS', 'PTCT', 'SRNE', 'ALT', 'DRRX', 'CVAC', 'AMPE', 'ANGN', 'CRVS', 'BMRA', 'BPTS', 'ARCT', 'HAE', 'RETA', 'ADPT', 'VIR', 'MMM', 'FULC', 'SNN', 'CLBS', 'VRNA', 'RDY', 'JAZZ', 'SAGE', 'MREO', 'IONS', 'KMDA', 'OPK', 'EW', 'ILMN', 'VXRT', 'DFFN', 'HRC', 'RIGL', 'NGM', 'AMGN', 'CFRX', 'NRBO', 'EQ', 'ONCS', 'NKTR', 'ICLR', 'ABIO', 'ENLV', 'EBS', 'TLC', 'LHDX', 'MDT', 'ADMP', 'VBLT', 'PRGO', 'BDX', 'AEMD', 'AMRN', 'AUPH', 'AVIR', 'BHC', 'CAPR', 'CLNN', 'DVA', 'EDSA', 'EVLO', 'GRTS', 'GRTX', 'HRTX', 'HSTO', 'HUGE', 'IMAB', 'JNJ', 'KPTI', 'LGVN', 'PLRX', 'PRTC', 'VAR', 'VBIV', 'XAIR']
    print('x symbol#:',len(xlist), 'w symbol#:', len(wlist))
    print('x missing:', set(wlist)-set(xlist))
    print('w missing:', set(xlist)-set(wlist))

In [46]:
# check missing symbols
if (debug):
    troublesymbols=set(wlist)-set(xlist)
    for ticker in list(troublesymbols)[1:]:
        print(ticker_all[ticker_all.Symbol==ticker])
        #ToDo: find company name from the symbol
        print(df[['lead_sponsor','collaborators', 'company','symbol']][df.company.str.find('Sanofi')==0])

In [47]:
# check extra symbols
if (debug):
    troublesymbols=set(xlist)-set(wlist)
    for ticker in list(troublesymbols)[1:]:
        print(ticker_all[ticker_all.Symbol==ticker])
        print(df[['sponsor_collaborators', 'company','symbol']][df.symbol==ticker])        

In [48]:
# filter out companies have no symbol to keep only public traded companies
df = df.dropna(subset=['symbol'])

# convert all the date to datetime
df['start_date']=pd.to_datetime(df['start_date'])
df['first_posted']=pd.to_datetime(df['first_posted'])
df['primary_completion_date']=pd.to_datetime(df['primary_completion_date'])
df['completion_date']=pd.to_datetime(df['completion_date'])
df['first_posted']=pd.to_datetime(df['first_posted'])
df['results_first_posted']=pd.to_datetime(df['results_first_posted'])
df['last_update_posted']=pd.to_datetime(df['last_update_posted'])

#df = df[(df['symbol'] !="") & (df['symbol'] !="nan")]
if (debug):
    print('final dataset after merge:',df.shape)
    print('total unique symbols:', df.symbol.nunique())
    print('symbol list', df.symbol.unique())

In [49]:
df.to_csv('all-merged-data.csv')